# Saving and Loading Tutorial

## Preparing a virtual environment

First, you need to have `Python3` and `openmpi` installed and running on your machine.

In a new directory, here are the steps I took to create a virtual environment for this Jupyter notebook:

	echo "" 
	echo "Preparing a virtual environment for NetPyNE" 
	echo "============================================================================="
	echo "Using Python version:"
	python3 --version
	echo "Using Python from:"
	which python3
	
	echo ""
	echo "Creating a virtual environment: python3 -m venv env"
	echo "-----------------------------------------------------------------------------"
	python3 -m venv env
	
	echo ""
	echo "Activating virtual environment: source env/bin/activate"
	echo "-----------------------------------------------------------------------------"
	source env/bin/activate
	
	echo ""
	echo "Updating pip: python3 -m pip install --upgrade pip"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install --upgrade pip

	echo ""
	echo "Installing wheel: python3 -m pip install --upgrade wheel"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install --upgrade wheel
	
	echo ""
	echo "Installing ipython: python3 -m pip install --upgrade ipython"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install ipython
	
	echo ""
	echo "Installing NEURON: python3 -m pip install --upgrade neuron"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install --upgrade neuron 
	
	echo ""
	echo "Cloning NetPyNE: git clone https://github.com/Neurosim-lab/netpyne.git"
	echo "-----------------------------------------------------------------------------"
	git clone https://github.com/Neurosim-lab/netpyne.git 
	
	echo ""
	echo "Installing NetPyNE: python3 -m pip install -e netpyne"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install -e netpyne 
	
	echo ""
	echo "Installing ipykernel for Jupyter: python3 -m pip install --upgrade ipykernel"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install --upgrade ipykernel 
	
	echo ""
	echo "Installing Jupyter: python3 -m pip install --upgrade jupyter"
	echo "-----------------------------------------------------------------------------"
	python3 -m pip install --upgrade jupyter
	
	echo ""
	echo "Creating a kernel for Jupyter: ipython kernel install --user --name=env"
	echo "-----------------------------------------------------------------------------"
	ipython kernel install --user --name=env

	echo ""
	echo "============================================================================="
	echo "Your virtual environment is ready for use."
	echo ""
	echo "To deactivate, execute: deactivate"
	echo "To reactivate, execute: source env/bin/activate"
	echo "============================================================================="

## Copying this tutorial

For convenience, let's copy this tutorial's directory up to the directory we're working in and then change into that directory.

    pwd
    cp -r netpyne/netpyne/tutorials/saving_loading_tut .
    cd saving_loading_tut
    pwd

## Normal saving

Then we'll run a simulation with normal saving, using `saving_netParams.py` (which is used by all simulations in this tutorial), `saving_normal_cfg.py`, and `saving_normal_init.py`.

Let's take a look at `saving_normal_init.py`, to see the standard way to run and save a simulation:

    from netpyne import sim

    cfg, netParams = sim.readCmdLineArgs(
        simConfigDefault='saving_normal_cfg.py', 
        netParamsDefault='saving_netParams.py')
    sim.initialize(simConfig=cfg, netParams=netParams)
    sim.net.createPops()
    sim.net.createCells()
    sim.net.connectCells()
    sim.net.addStims()
    sim.setupRecording()
    sim.runSim()
    sim.gatherData()
    sim.saveData()
    sim.analysis.plotData()
    
We could run this on a single core using `python3 saving_normal_init.py` (if we just want the output) or `ipython -i saving_normal_init.py` (if we wanted to interact with the simulation afterwards.  But we will run this on multiple cores using the following command:

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_normal_init.py

This command does not currently exit to the system prompt, so you will have to restart your kernel.  In the menu bar above, click on `Kernel`, then `Restart`, then `Restart`.

The `whos` in the next cell should return `Interactive namespace is empty.` after the Kernel has been cleared.

In [ ]:
whos

The simulation should have produced a directory called `saving_normal_data` with three analysis plots and a data file named `saving_normal_data.pkl`.  We are now going to load the simulation from this file and produce the same plots.

In [ ]:
from netpyne import sim
sim.loadAll('saving_normal_data/saving_normal_data.pkl')

In [ ]:
sim.analysis.plotConn(saveFig='saving_normal_data/saving_normal_plot_conn_pop_strength_matrix_FROMFILE.png');
sim.analysis.plotRaster(saveFig='saving_normal_data/saving_normal_raster_gid_FROMFILE.png');
sim.analysis.plotTraces(saveFig='saving_normal_data/saving_normal_traces_FROMFILE.png');

Compare the plots, they should be identical.  Congratulations!  You have run a simulation, saved the data, then loaded it later to perform more analysis.

Now restart your kernel and check the `whos`.

In [ ]:
whos

## Distributed Saving

If you're running large sims, you may want to save the data from each node in a separate file, i.e. distributed saving.

We'll run a simulation using distributed saving and loading using `saving_netParams.py` (which is used by all simulations in this tutorial), `saving_dist_cfg.py`, and `saving_dist_init.py`.

The only changes to the cfg file are renaming the simulation:

    cfg.simLabel = 'saving_dist'

and turning off the saving of the data into one file:

    cfg.savePickle = False #True

Our init file for distributed saving looks like this:

from netpyne import sim

    cfg, netParams = sim.readCmdLineArgs(
        simConfigDefault='saving_dist_cfg.py', 
        netParamsDefault='saving_netParams.py')
    sim.initialize(simConfig=cfg, netParams=netParams)
    sim.net.createPops()
    sim.net.createCells()
    sim.net.connectCells()
    sim.net.addStims()
    sim.setupRecording()
    sim.runSim()
    #sim.gatherData()
    #sim.saveData()
    ##### new #####
    sim.saveDataInNodes()
    sim.gatherDataFromFiles()
    ##### end new #####
    sim.analysis.plotData()
    
We turned off `gatherData` and `saveData` and replaced those with `saveDataInNodes` and `gatherDataFromFiles`.

Let's run the simulation now.

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_dist_init.py

That should have produced a directory `saving_dist_data` containing the same three analysis plots and a `node_data` directory containing a data file from each of the four nodes we used.

Now restart your kernel so we can load the data from file analyze it again.

The `whos` in the next cell should return `Interactive namespace is empty.`

In [ ]:
whos

In [ ]:
from netpyne import sim
sim.gatherDataFromFiles(simLabel='saving_dist')

In [ ]:
sim.analysis.plotConn(saveFig='saving_dist_data/saving_dist_plot_conn_pop_strength_matrix_FROMFILE.png');
sim.analysis.plotRaster(saveFig='saving_dist_data/saving_dist_raster_gid_FROMFILE.png');
sim.analysis.plotTraces(saveFig='saving_dist_data/saving_dist_traces_FROMFILE.png');

Compare the plots, they should be identical except for the connectivity plot, which didn't retain the connectivity for the background inputs.

Now restart your kernel and check the `whos`.

In [ ]:
whos

## Interval Saving

Perhaps you want to save data at intervals in case you have large, long simulations you're worried won't complete.

We'll run a simulation using interval saving and loading using `saving_netParams.py` (which is used by all simulations in this tutorial), `saving_int_cfg.py`, and `saving_int_init.py`.

The only changes to the cfg file are renaming the simulation:

    cfg.simLabel = 'saving_int'

and turning back on the saving of the data into one file:

    cfg.savePickle = True

Our init file for interval saving looks like this:

from netpyne import sim

    from netpyne import sim

    cfg, netParams = sim.readCmdLineArgs(
        simConfigDefault='saving_int_cfg.py', 
        netParamsDefault='saving_netParams.py')
    sim.initialize(simConfig=cfg, netParams=netParams)
    sim.net.createPops()
    sim.net.createCells()
    sim.net.connectCells()
    sim.net.addStims()
    sim.setupRecording()
    #sim.runSim()
    ##### new #####
    sim.runSimIntervalSaving(1000)
    ##### end new #####
    sim.gatherData()
    sim.saveData()
    sim.analysis.plotData()
    
We turned off `runSim` and replaced it with `runSimIntervalSaving(1000)`, which will save the simulation every 1000 ms.

Let's run the simulation now.  Remember you can run this without MPI using the command `python3 saving_int_init.py`.

In [ ]:
!mpiexec -n 4 nrniv -python -mpi saving_int_init.py

That should have produced a directory `saving_int_data` containing the data file and the same three analysis plots (from the completed simulation) and an `interval_data` directory containing a data file for each 1000 ms of our 10,000 ms simulation.

Now restart your kernel so we can load interval data from file.

The `whos` in the next cell should return `Interactive namespace is empty.`

In [ ]:
whos

Now, let's assume our simulation timed out, and the last interval save we got was at 5000 ms.  We can still analyze that partial data.

In [ ]:
from netpyne import sim
sim.loadAll('saving_int_data/interval_data/interval_5000.pkl', createNEURONObj=False)

In [ ]:
sim.analysis.plotConn(saveFig='saving_int_data/saving_int_plot_conn_pop_strength_matrix_INTERVAL.png');
sim.analysis.plotRaster(saveFig='saving_int_data/saving_int_raster_gid_INTERVAL.png');
sim.analysis.plotTraces(saveFig='saving_int_data/saving_int_traces_INTERVAL.png');

The connectivity plot should be identical and the raster plot is currently failing for interval saving (debugging in progress), but you can see that we recovered partial data from the traces plot.

Congratulations!  You have successfully saved, loaded, and analyzed simulation data in a variety of ways.